In [ ]:
cd//content/drive/MyDrive/Colab Notebooks/Assignment4

/content/drive/MyDrive/Colab Notebooks/Assignment4


In [ ]:
!unzip archive.zip

Archive:  archive.zip
replace spam.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


**Importing libraries**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow
import seaborn as sns
import matplotlib.pyplot as plt
import keras
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Assignment4/spam.csv',delimiter=',', encoding="ISO-8859-1") 

In [ ]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
data.shape

(5572, 5)

Preprocessing

In [ ]:
data=data.drop(columns=["Unnamed: 2","Unnamed: 3","Unnamed: 4"])

In [ ]:
data=data.rename({"v1":"Category","v2":"Message"},axis=1)

In [ ]:
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
data["Message Length"]=data["Message"].apply(len)

In [ ]:
ham_desc=data[data["Category"]=="ham"]["Message Length"].describe()
spam_desc=data[data["Category"]=="spam"]["Message Length"].describe()

In [ ]:
data.describe(include="all")

,Category,Message,Message Length
count,5572,5572,5572.000000
unique,2,5169,NaN
top,ham,"Sorry, I'll call later",NaN
freq,4825,30,NaN
mean,NaN,NaN,80.118808
std,NaN,NaN,59.690841
min,NaN,NaN,2.000000
25%,NaN,NaN,36.000000
50%,NaN,NaN,61.000000
75%,NaN,NaN,121.000000


In [ ]:
data["Category"].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [ ]:
minority_len=len(data[data["Category"]=="spam"])
majority_len=len(data[data["Category"]=="ham"])

In [ ]:
minority_indices=data[data["Category"]=="spam"].index
majority_indices=data[data["Category"]=="ham"].index

In [ ]:
random_majority_indices=np.random.choice(majority_indices,size=minority_len,replace=False)

In [ ]:
undersampled_indices=np.concatenate([minority_indices,random_majority_indices])

In [ ]:
df=data.loc[undersampled_indices]

In [ ]:
df=df.sample(frac=1)

In [ ]:
df=df.reset_index()

In [ ]:
df=df.drop(columns=["index"],)

We balanced our ham and spam data

In [ ]:
df["Category"].value_counts()

ham     747
spam    747
Name: Category, dtype: int64

In [ ]:
df["Label"]=df["Category"].map({"ham":0,"spam":1})

In [ ]:
df.head()

,Category,Message,Message Length,Label
0,ham,Your gonna have to pick up a $1 burger for you...,108,0
1,ham,"Yo, any way we could pick something up tonight?",47,0
2,spam,Please CALL 08712402972 immediately as there i...,81,1
3,ham,"Not much, just some textin'. How bout you?",42,0
4,ham,O i played smash bros &lt;#&gt; religiously.,46,0


In [ ]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()

**Create model**

In [ ]:
   
corpus=[]

#iterate through the df["Message"]
for message in df["Message"]:

    
    #replace every special characters, numbers etc.. with whitespace of message
    #It will help retain only letter/alphabets
  message=re.sub("[^a-zA-Z]"," ",message)
    
    #convert every letters to its lowercase
  message=message.lower()
  message=message.split()
    
    #perform stemming using PorterStemmer for all non-english-stopwords
  message=[stemmer.stem(words)
            for words in message
             if words not in set(stopwords.words("english"))
            ]
    #join the word lists with the whitespace
  message=" ".join(message)
    
    #append the message in corpus list
  corpus.append(message)

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
vocab_size=10000
oneHot_doc=[one_hot(words,n=vocab_size)
           for words in corpus]

In [ ]:
df["Message Length"].describe()

count    1494.000000
mean      104.522758
std        57.435568
min         2.000000
25%        49.250000
50%       117.000000
75%       153.000000
max       632.000000
Name: Message Length, dtype: float64

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sentence_len=200
embedded_doc=pad_sequences(oneHot_doc,maxlen=sentence_len,padding="pre")

In [ ]:
extract_features=pd.DataFrame(data=embedded_doc)
target=df["Label"]

In [ ]:
df_final=pd.concat([extract_features,target],axis=1)

In [ ]:
#split the dataframe into dependent and independent variables
X=df_final.drop("Label",axis=1)
y=df_final["Label"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_trainval,X_test,y_trainval,y_test=train_test_split(X,y,random_state=42,test_size=0.15)

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X_trainval,y_trainval,random_state=42,test_size=0.15)

Build the model

In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential

In [ ]:
model=Sequential()

In [ ]:
feature_num=100
model.add(Embedding(input_dim=vocab_size,output_dim=feature_num,input_length=sentence_len))
model.add(LSTM(units=128))
model.add(Dense(units=1,activation="sigmoid"))

Compile the model

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),loss="binary_crossentropy",metrics=["accuracy"])

fit the model

In [ ]:
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=10)

Epoch 1/10
34/34 [==============================] - 12s 285ms/step - loss: 0.5037 - accuracy: 0.7950 - val_loss: 0.3403 - val_accuracy: 0.8953
Epoch 2/10
34/34 [==============================] - 13s 370ms/step - loss: 0.1742 - accuracy: 0.9481 - val_loss: 0.1464 - val_accuracy: 0.9581
Epoch 3/10
34/34 [==============================] - 8s 245ms/step - loss: 0.0619 - accuracy: 0.9833 - val_loss: 0.0885 - val_accuracy: 0.9738
Epoch 4/10
34/34 [==============================] - 7s 213ms/step - loss: 0.0280 - accuracy: 0.9917 - val_loss: 0.0832 - val_accuracy: 0.9738
Epoch 5/10
34/34 [==============================] - 7s 216ms/step - loss: 0.0128 - accuracy: 0.9972 - val_loss: 0.0881 - val_accuracy: 0.9686
Epoch 6/10
34/34 [==============================] - 7s 216ms/step - loss: 0.0126 - accuracy: 0.9963 - val_loss: 0.0845 - val_accuracy: 0.9686
Epoch 7/10
34/34 [==============================] - 7s 214ms/step - loss: 0.0080 - accuracy: 0.9972 - val_loss: 0.0841 - val_accuracy: 0.9738
Epoc

In [ ]:
y_pred=model.predict(X_test)
y_pred=(y_pred>0.5)

8/8 [==============================] - 1s 53ms/step


test the model

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
score=accuracy_score(y_test,y_pred)
print("Test Score:{:.2f}%".format(score*100))

Test Score:96.44%


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def classify_message(model,message):
    for sentences in message:
        sentences=nltk.sent_tokenize(message)
        for sentence in sentences:
            #replace all special characters
            words=re.sub("[^a-zA-Z]"," ",sentence)
            if words not in set(stopwords.words('english')):
                word=nltk.word_tokenize(words)
                word=" ".join(word)
        oneHot=[one_hot(word,n=vocab_size)]
    text=pad_sequences(oneHot,maxlen=sentence_len,padding="pre")
    predict=model.predict(text)
    
    #if predict value is greater than 0.5 its a spam
    if predict>0.5:
        print("It is a spam")
    #else the message is not a spam    
    else:
        print("It is not a spam")

In [ ]:
message1="I am having a bad day and I would like to have a break today"
message2="This is to inform you had won a lottery and the subscription will end in a week so call us."

In [ ]:
import nltk

In [ ]:
classify_message(model,message1)

1/1 [==============================] - 0s 22ms/step
It is not a spam


In [ ]:
classify_message(model,message2)

1/1 [==============================] - 0s 18ms/step
It is a spam
